# Flag Pattern Detection

In [1]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_01.07.2020-15.07.2023.csv")
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,time,open,high,low,close,volume
0,01.07.2020 00:00:00.000,1.12336,1.12336,1.12275,1.12306,4148.0298
1,01.07.2020 01:00:00.000,1.12306,1.12395,1.12288,1.12385,5375.5801
2,01.07.2020 02:00:00.000,1.12386,1.12406,1.12363,1.12382,4131.6099
3,01.07.2020 03:00:00.000,1.12382,1.12388,1.12221,1.12265,4440.6001
4,01.07.2020 04:00:00.000,1.12265,1.12272,1.12151,1.12179,4833.1001
5,01.07.2020 05:00:00.000,1.12179,1.12261,1.12156,1.12240,6689.5601
6,01.07.2020 06:00:00.000,1.12240,1.12343,1.12202,1.12333,7562.7500
7,01.07.2020 07:00:00.000,1.12331,1.12331,1.12231,1.12315,8641.7500
8,01.07.2020 08:00:00.000,1.12315,1.12448,1.12290,1.12311,10042.7695
9,01.07.2020 09:00:00.000,1.12313,1.12337,1.12076,1.12076,9587.4004


In [2]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,3,3), axis=1)

## Pivot Candles Vizualisation

In [3]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[500:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [5]:
from scipy.stats import linregress

def detect_flag(candle, backcandles, window, plot_flag=False):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['pivot'] == 2].high.tail(3).values
    idxhighs = localdf[localdf['pivot'] == 2].high.tail(3).index
    lows = localdf[localdf['pivot'] == 1].low.tail(3).values
    idxlows = localdf[localdf['pivot'] == 1].low.tail(3).index

    if len(highs) == 3 and len(lows) == 3:
        order_condition = ( 
            (idxlows[0] < idxhighs[0] 
            < idxlows[1] < idxhighs[1] 
            < idxlows[2] < idxhighs[2]) 
            or 
            (idxhighs[0] < idxlows[0] 
             < idxhighs[1] < idxlows[1] 
             < idxhighs[2] < idxlows[2]) )
        
        slmin, intercmin, rmin, _, _ = linregress(idxlows, lows)
        slmax, intercmax, rmax, _, _ = linregress(idxhighs, highs)

        if (order_condition 
            and (rmax*rmax)>=0.9 
            and (rmin*rmin)>=0.9 
            and slmin>=0.0001 
            and slmax<=-0.0001):
            #and ((abs(slmin)-abs(slmax))/abs(slmax)) < 0.05):

            if plot_flag:
                fig = go.Figure(data=[go.Candlestick(x=localdf.index,
                open=localdf['open'],
                high=localdf['high'],
                low=localdf['low'],
                close=localdf['close'])])

                fig.add_scatter(x=localdf.index, y=localdf['pointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot")
                fig.add_trace(go.Scatter(x=idxlows, y=slmin*idxlows + intercmin, mode='lines', name='min slope'))
                fig.add_trace(go.Scatter(x=idxhighs, y=slmax*idxhighs + intercmax, mode='lines', name='max slope'))
                fig.update_layout(
                xaxis_rangeslider_visible=False,
                plot_bgcolor='white', # change the background to white
                xaxis=dict(showgrid=True, gridcolor='white'), # change the x-axis grid to white
                yaxis=dict(showgrid=True, gridcolor='white') # change the y-axis grid to white
                )
                fig.show()

            return 1
    
    return 0

In [6]:
detect_flag(2189, 30, 3, plot_flag=True)

1

In [7]:
df['flag'] = df.index.map(lambda x: detect_flag(x, 35, 3))

In [8]:
df[df['flag']!=0]

,time,open,high,low,close,volume,pivot,pointpos,flag
2189,05.11.2020 06:00:00.000,1.17361,1.17426,1.17343,1.17351,28825.810500,0,NaN,1
2190,05.11.2020 07:00:00.000,1.17351,1.17487,1.17274,1.17457,60948.949200,1,1.17174,1
2191,05.11.2020 08:00:00.000,1.17455,1.17699,1.17402,1.17629,59373.828100,0,NaN,1
2192,05.11.2020 09:00:00.000,1.17628,1.18023,1.17551,1.17943,48626.531300,0,NaN,1
2193,05.11.2020 10:00:00.000,1.17942,1.18114,1.17887,1.18113,48942.070300,0,NaN,1
14156,25.11.2022 00:00:00.000,1.04074,1.04113,1.03959,1.04055,0.016456,1,1.03859,1
14157,25.11.2022 01:00:00.000,1.04056,1.04148,1.04034,1.04094,0.021063,0,NaN,1
14158,25.11.2022 03:00:00.000,1.04110,1.04183,1.04045,1.04159,0.013943,0,NaN,1
14159,25.11.2022 05:00:00.000,1.04213,1.04220,1.04071,1.04110,0.016415,0,NaN,1
